In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import folium
import matplotlib.pyplot as plt
import calendar
from folium.plugins import FloatImage

## Reading the data

In [62]:
data14=pd.read_csv("Chicago_Crimes_2001_to_2004.csv",error_bad_lines=False)
data57=pd.read_csv("Chicago_Crimes_2005_to_2007.csv",error_bad_lines=False)
data811=pd.read_csv("Chicago_Crimes_2008_to_2011.csv",error_bad_lines=False)
data1217=pd.read_csv("Chicago_Crimes_2012_to_2017.csv",error_bad_lines=False)

b'Skipping line 1513591: expected 23 fields, saw 24\n'
C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (17,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
b'Skipping line 533719: expected 23 fields, saw 24\n'
b'Skipping line 1149094: expected 23 fields, saw 41\n'


In [63]:
print("total number of rows:",data14.shape[0]+data57.shape[0]+data811.shape[0]+data1217.shape[0])
print("total number of columns:",data14.shape[1])

total number of rows: 7941282
total number of columns: 23


In [77]:
data14 = data14.drop(columns=['Unnamed: 0', 'ID', 'Case Number','IUCR','Domestic', 'Beat', 'District','Arrest', 'Ward','X Coordinate', 'Y Coordinate','Latitude','Longitude','Community Area','Updated On', 'FBI Code'], axis = 1)
data57 = data57.drop(columns=['Unnamed: 0', 'ID', 'Case Number','IUCR','Domestic', 'Beat', 'District','Arrest', 'Ward','X Coordinate', 'Y Coordinate','Latitude','Longitude','Community Area','Updated On', 'FBI Code'], axis = 1)
data811 = data811.drop(columns=['Unnamed: 0', 'ID', 'Case Number','IUCR','Domestic', 'Beat', 'District','Arrest', 'Ward','X Coordinate', 'Y Coordinate','Latitude','Longitude','Community Area','Updated On', 'FBI Code'], axis = 1)
data1217 = data1217.drop(columns=['Unnamed: 0', 'ID', 'Case Number','IUCR','Domestic', 'Beat', 'District','Arrest', 'Ward','X Coordinate', 'Y Coordinate','Latitude','Longitude','Community Area','Updated On', 'FBI Code'], axis = 1)

In [78]:
data14=data14.dropna()  ## dropping rows with missing data
data57=data57.dropna()
data811=data811.dropna()
data1217=data1217.dropna()

In [79]:
data14["Year"]=data14["Year"].astype(int)
data57["Year"]=data57["Year"].astype(int)
data811["Year"]=data811["Year"].astype(int)
data1217["Year"]=data1217["Year"].astype(int)

In [80]:
data14.head()

,Date,Block,Primary Type,Description,Location Description,Year,Location
1,03/01/2003 12:00:00 AM,004XX W 42ND PL,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,2003,"(41.817229156, -87.637328162)"
4,05/01/2003 01:00:00 AM,111XX S NORMAL AVE,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,2003,"(41.691784636, -87.635115968)"
6,01/01/2001 11:00:00 AM,114XX S ST LAWRENCE AVE,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,2001,"(41.687020002, -87.60844523)"
7,03/15/2003 12:00:00 AM,090XX S RACINE AVE,OTHER OFFENSE,OTHER WEAPONS VIOLATION,RESIDENCE PORCH/HALLWAY,2003,"(41.729712374, -87.653158513)"
9,01/01/2003 12:00:00 AM,009XX S SPAULDING AVE,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,2003,"(41.869772159, -87.708180162)"


### Exporting files for analysis

In [7]:
data14.to_csv('Chicago_Crimes_2001_to_2004_pre.csv',sep='\t')
data57.to_csv('Chicago_Crimes_2005_to_2007_pre.csv',sep='\t')
data811.to_csv('Chicago_Crimes_2008_to_2011_pre.csv',sep='\t')
data1217.to_csv('Chicago_Crimes_2012_to_2017_pre.csv',sep='\t')

In [8]:
print("total number of rows after preprocessing:",data14.shape[0]+data57.shape[0]+data811.shape[0]+data1217.shape[0])
print("total number of columns after preprocessing:",data14.shape[1])

total number of rows after preprocessing: 7834222
total number of columns after preprocessing: 7


##  Mapping

In [9]:
def adding_labels(df,a):
    x=[]
    y=[]
    for i in range(df.shape[0]):
        x.append(df.iloc[i][0])
        y.append(df.iloc[i][1])
    df = pd.DataFrame(list(zip(x, y,[a]*len(x))),columns =['Lat', 'Long','Label'])
    df.head()  
    return df

In [13]:
robbery=adding_labels(pd.read_csv("Robbery.csv",header=None),1)
assault=adding_labels(pd.read_csv("Assasult.csv",header=None),2)
narcotics=adding_labels(pd.read_csv("Narco.csv",header=None),3)

In [20]:
frames = [narcotics[:500], assault[:500],robbery[:500]]
result = pd.concat(frames)  

In [74]:
def plot_map(df,image_file):
    chicago_map_crime = folium.Map(location=[41.779190321,-87.682688805],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

    for i in range(df.shape[0]):
        lat = df.iloc[i][0]
        long = df.iloc[i][1]
        radius = 0.5
        if(df["Label"].iloc[i]==1):
            color = "#003EFF"
        elif(df["Label"].iloc[i]==2): 
            color = "#BFFF00"
        else:
            color = "#FF0000"
        folium.CircleMarker(location = [lat, long],radius = radius, color = color, fill = True).add_to(chicago_map_crime)
        FloatImage(image_file, bottom=80, left=65).add_to(chicago_map_crime)
    return  chicago_map_crime

In [75]:
image_file = 'Capture.PNG'
plot_map(result,image_file)

In [30]:
def individual_maps(df):
    chicago_map_crime = folium.Map(location=[41.779190321,-87.682688805],
                            zoom_start=13,
                            tiles="CartoDB dark_matter")
    for i in range(df.shape[0]):
        lat = df.iloc[i][0]
        long = df.iloc[i][1]
        radius = 1
        color = "#003EFF"
        folium.CircleMarker(location = [lat, long],radius = radius, color = color, fill = True).add_to(chicago_map_crime)
    return chicago_map_crime

In [31]:
school=pd.read_csv("School.csv",header=None)
individual_maps(school)

In [33]:
blocks=pd.read_csv("Location_count.csv",header=None)
blocks.columns = ["Lat", "Long", "Count"]

In [47]:
chicago_map_crime = folium.Map(location=[41.976290,-87.905227],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

for i in range(blocks.shape[0]):
    lat = blocks.iloc[i][0]
    long = blocks.iloc[i][1]
    radius = 1
    if(blocks["Count"].iloc[i]>5000):
        color = "#FF0000"
    elif(5000>=blocks["Count"].iloc[i]>2500): 
        color = "#FF8C00"
    elif(2500>=blocks["Count"].iloc[i]>=1000): 
        color = "#FFFF00"
    folium.CircleMarker(location = [lat, long],radius = radius, color = color, fill = True).add_to(chicago_map_crime)
chicago_map_crime

## Bogus check analysis

In [48]:
def findDay(date):
    month,day, year = (int(i) for i in date.split('/'))    
    dayNumber = calendar.weekday(year, month, day)
    days =["Monday", "Tuesday", "Wednesday", "Thursday","Friday", "Saturday", "Sunday"]
    return (days[dayNumber])

In [49]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [57]:
bogus_check=pd.read_csv("bogus_check_date.csv",header=None)
bogus_check.head()

,0,1
0,12/29/2006,22
1,01/02/2001,21
2,02/23/2001,21
3,02/16/2001,19
4,01/10/2001,19


In [54]:
lis=[]
bogus_check=bogus_check[bogus_check.columns[0]]
for i in bogus_check:
    lis.append(findDay(i))

In [58]:
most_frequent(lis[0:5])

'Friday'